In [ ]:
from cedne import cedne
from cedne import utils
from matplotlib import pyplot as plt
from networkx.algorithms.community import modularity
from collections import Counter
import numpy as np
import scipy.stats as ss

In [ ]:
w = utils.makeWorm(chem_only=True)
nn = w.networks["Neutral"]
utils.loadNeuropeptides(w)

In [ ]:
girvan_newman = utils.nx.algorithms.community.girvan_newman

In [ ]:
type(w)

In [ ]:
print(len(w.networks))

In [ ]:
comms = list(girvan_newman(nn))

In [ ]:
def calculate_cut(S, G):
    """Calculate the cut between S and its complement."""
    return sum(1 for u in S for v in G if v not in S and G.has_edge(u, v))

def calculate_vol(S, G):
    """Calculate the volume of a set S."""
    return sum(G.degree(n) for n in S)

def energy(nn, comm):
    ''' Calculates the energy for a community as the number of within community edges minus the number of between community edges.'''
    tot_edges = len(nn.edges)
    within = 0
    between = 0
    for i,n1 in enumerate(comm):
        sub1 = nn.subnetwork(neuron_names = [n.name for n in n1], as_view=True)
        sub1_edges = len(sub1.edges)
        within += sub1_edges
    between = tot_edges - within
    return within - between


In [ ]:
energy_comm = [energy(nn, comm) for comm in comms]

In [ ]:
def is_valid_partition(G, partition):
    """Check if a partition is valid for a graph G."""
    nodes_in_partition = set(node for subset in partition for node in subset)
    return set(G.nodes()) == nodes_in_partition and all(
        len(set(subset)) == len(subset) for subset in partition
    )

In [ ]:
mod_score= []
for comm in comms:
    communities = [list(c) for c in comm]
    print(is_valid_partition(nn, communities))
    #mod_score.append(modularity(nn, communities))

In [ ]:
comm_num = 18
f, ax = plt.subplots(figsize=(6,6), layout='constrained')
ax.plot(energy_comm, color='k')
ax.axvline(x=comm_num, color='r', linestyle='--')
ax.set_xlabel("Community Number")
ax.set_ylabel("Energy")
utils.simpleaxis(ax)
plt.show()

In [ ]:
comm = comms[comm_num]
for j,c in enumerate(comm):
    print(j, sorted([n.name for n in c]))

In [ ]:
betweenness = utils.nx.betweenness_centrality(nn)

In [ ]:
for n, b in sorted(betweenness.items(), key=lambda x: x[1], reverse=True):
    print(n.name, b)

In [ ]:
nc = utils.nx.edge_connectivity(nn)

In [ ]:
from networkx.algorithms.centrality import percolation_centrality
percolation = percolation_centrality(nn)

In [ ]:
for n, p in sorted(percolation.items(), key=lambda x: x[1], reverse=True):
    print(n.name, p)

In [ ]:
conn_np = {n:[] for n in nn.neurons}
for neup, net in w.networks.items():
    if not neup == 'Neutral':
        for n in nn.neurons:
            conn_np[n].append([o.name for (_,o,_) in net.neurons[n].get_connections(direction='out').keys()])

In [ ]:
adj = {}
for n in conn_np.keys():
    down_conns = [m for nlist in conn_np[n] for m in nlist]
    adj[n] = {k:{'weight':weight} for k,weight in Counter(down_conns).items()}

In [ ]:
nn_np = cedne.NervousSystem(worm=w, network='NP_summary')
nn_np.create_neurons_from(nn, data=True)
nn_np.setup_connections(adjacency=adj, connection_type='NP')

In [ ]:
percolation_np = percolation_centrality(nn_np)

In [ ]:
nnames, chemical, neuropep = [], [], []
for n, p in sorted(percolation.items(), key=lambda x: x[1], reverse=True):
    # if p>0 and percolation_np[nn_np.neurons[n.name]]>0 :
        nnames.append(n.name)
        chemical.append(p)
        neuropep.append(percolation_np[nn_np.neurons[n.name]])

In [ ]:
nclasses = [nn.neurons[n].category for n in nnames]
class_list = list(set(nclasses))
color_list = plt.cm.rainbow(np.linspace(0,1,len(class_list)))
colors = [color_list[class_list.index(n)] for n in nclasses]
color_1 = 'k'

chemar, neuropepar = np.log10(chemical), np.log10(neuropep)

f, ax = plt.subplots(figsize=(3,3))
ax.scatter(chemar, neuropepar, color=color_1, s=12)
res = ss.linregress(chemar, neuropepar)
ax.plot(np.array(chemar), res.slope*np.array(chemar) + res.intercept, ls = '--')
print(res)
# ax.set_xscale('log')
# ax.set_yscale('log')
# ax.set_xlim((10**-6, 10**-1))
# ax.set_ylim((10**-6, 10**-1))
utils.simpleaxis(ax)
plt.show()

In [ ]:
for c,n in zip(chemar, neuropepar):
    print(c,n)

In [ ]:
out_conn_chem = []
out_conn_np = []

for n in nn.neurons:
    out_conn_chem.append(len(nn.neurons[n].get_connections(direction='both')))
    out_conn_np.append(len(nn_np.neurons[n].get_connections(direction='both')))

In [ ]:
f, ax = plt.subplots(figsize=(3,3))
ax.scatter(out_conn_chem, out_conn_np, color=color_1, s=12)
# ax.set_xscale('log')
# ax.set_yscale('log')
# ax.set_xlim((1, 10**2.5))
# ax.set_ylim((1, 10**2.5))
utils.simpleaxis(ax)
plt.show()

In [ ]:
npthres= 350
f, ax = plt.subplots(figsize=(2,2))
ax.hist(out_conn_np, bins=50, color='gray')
ax.axvline(x=npthres, ls = '--', color='k')
utils.simpleaxis(ax)
plt.show()

In [ ]:
nodes_nphigh = sorted([nnames[k] for k in np.where(np.array(out_conn_np)>npthres)[0]])

In [ ]:
nn_np_folded = utils.foldByNeuronType(nn_np)

In [ ]:
edges = list(nn_np_folded.connections.keys())

In [ ]:
fig_ent = utils.plot_shell(nn_np_folded, save='neuropep-grouped.svg', shells=[[], [nn_np_folded.neurons[n] for n in nn_np_folded.neurons]], figsize=(8,8), width_logbase=10)

In [ ]:
nn